# Convolutional Neural Networks

- Convolutional Layer 안에는 kernel이 있음
  - kernel (Computer Vision 용어) : weight와 bias라고 생각하면 됨
- Neural Network 만들면 그것에 대한 테이블(그림)로 반드시 표현해 줄 것 ★★
  - 나중에 검증하고, 설계할 때 아주 유용하게 사용할 수 있음
- Tensorflow는 매우 유연함
  - 같은 것을 다양한 방법으로 구현 가능
  - 아래와 같이 각각의 구현하는 방법의 장단점을 분명하게 알고, 원할 때, 적절하게 사용하기 ★

# 1. Shapes in CNNs

아래 코드가 Convolutional Neural Network의 Forward Proagation의 전부
- 가장 간단한 모델을 만들기 위한 최소한의 지식

## 1.1 Shapes in the Feature Extractors

- convolutional layer, pooling layer의 반복을 통해 만들어짐
- 마지막에는 Feature vector 생성 : (32, 245)

In [7]:
import tensorflow as tf

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

# channel : 3 - color image
N, n_H, n_W, n_c = 32, 28, 28, 3
n_conv_filter = 5
k_size = 3
pool_size, pool_strides = 2, 2
batch_size = 32

# normal distribution에서 임의의 noise가 담긴 이미지 N(32)개 생성
x = tf.random.normal(shape=(N, n_H, n_W, n_c))

conv1 = Conv2D(filters=n_conv_filter, kernel_size=k_size,
              padding='same', activation='relu')

conv1_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

conv2 = Conv2D(filters=n_conv_filter, kernel_size=k_size,
              padding='same', activation='relu')

conv2_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

flatten = Flatten()

print("Input: {}\n".format(x.shape))

Input: (32, 28, 28, 3)



max pooling layer는 trainable parameter(weight와 bias)가 들어있지 않음 

In [8]:
x = conv1(x)
W, B = conv1.get_weights()
print("W/B: {}/{}".format(W.shape, B.shape))
print("After conv1: {}".format(x.shape))  # same padding → 28 * 28 고정, neuron 개수 : 5개
x = conv1_pool(x)
print("After conv1_pool: {}\n".format(x.shape)) # 2칸씩 건너 뛰었으니 (strides) 사이즈가 절반이 됨 → 14 * 14

W/B: (3, 3, 3, 5)/(5,)
After conv1: (32, 28, 28, 5)
After conv1_pool: (32, 14, 14, 5)



weight 값 의미 (3, 3, 5, 5)
- 3 * 3
- 세 번째 5 : 위에서 conv1 pooling 지나며, channel이 5개가 되었었음 : input = 5
- 네 번째 5 : filter = 5

In [10]:
x = conv2(x)
W, B = conv2.get_weights()
print("W/B: {}/{}".format(W.shape, B.shape))
print("After conv2: {}".format(x.shape))
x = conv2_pool(x)
print("After conv2_pool: {}".format(x.shape)) # 2칸씩 건너 뛰었으니 (strides) 사이즈가 절반이 됨 → 7 * 7

x = flatten(x)
print("After flatten: {}".format(x.shape))   # 7 * 7 * 5 = 245

W/B: (3, 3, 5, 5)/(5,)
After conv2: (32, 14, 14, 5)
After conv2_pool: (32, 7, 7, 5)
After flatten: (32, 245)


## 1.2 Shapes in the Classifier

In [11]:
from tensorflow.keras.layers import Dense

n_neurons = [50, 25, 10]

dense1 = Dense(units=n_neurons[0], activation='relu')
dense2 = Dense(units=n_neurons[1], activation='relu')
dense3 = Dense(units=n_neurons[2], activation='softmax')  # 마지막 layer는 softmax

print("Input feature: {}\n".format(x.shape))

x = dense1(x)
W, B = dense1.get_weights()
print("W/B: {}/{}".format(W.shape, B.shape))  # 50개의 neuron 생성
print("After dense1: {}\n".format(x.shape))   # input의 32를 가져오고, 첫 번째 neuron 개수만큼 만들어짐 (50)

x = dense2(x)
W, B = dense2.get_weights()
print("W/B: {}/{}".format(W.shape, B.shape))
print("After dense2: {}\n".format(x.shape)) 

x = dense3(x)
W, B = dense3.get_weights()
print("W/B: {}/{}".format(W.shape, B.shape))
print("After dense3: {}\n".format(x.shape))    # 마지막 10개 class에 대해 각각의 probability vector 생성

Input feature: (32, 245)

W/B: (245, 50)/(50,)
After dense1: (32, 50)

W/B: (50, 25)/(25,)
After dense2: (32, 25)

W/B: (25, 10)/(10,)
After dense3: (32, 10)



## 1.3 Shapes in the Loss Functions

In [14]:
from tensorflow.keras.losses import CategoricalCrossentropy

y = tf.random.uniform(minval=0, maxval=10,  # 0~9 class
                      shape=(32, ),         # vector
                      dtype=tf.int32)

y = tf.one_hot(y, depth=10)
print(y)

tf.Tensor(
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]

In [16]:
loss_object = CategoricalCrossentropy()
loss = loss_object(y, x)   # x : predictions
print(loss.shape)
print(loss)

()
tf.Tensor(2.4431143, shape=(), dtype=float32)


# 2. Implementation of CNNS

전체 Convolutional Neural Network를 Tensorflow에서 제공하는 모델을 만드는 방법으로 만들어보려고 함

## 2.1 Implementation with Sequential Method

In [18]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

sequential model
- 가장 큰 장점 : .add를 이용해서 layer를 추가만 하면 되니 만들기 간편
- 단점
  - 중간에 sequential한 connection이 아니라 다른 식의 tensor 연산이 필요하면 사용 불가능
  - layer를 새로 하나 만들 때마다 코드가 길어짐

In [20]:
N, n_H, n_W, n_C = 4, 28, 28, 3
n_conv_neurons = [10, 20, 30]
n_dense_neurons = [50, 30, 10]
k_size = 3   # kernel size
padding = 'same'
pool_size, pool_strides = 2, 2
activation = 'relu'

x = tf.random.normal(shape=(N, n_H, n_W, n_C))

print("Input : ", x.shape)

model = Sequential()

model.add(Conv2D(filters=n_conv_neurons[0],
                 kernel_size=k_size, padding=padding, activation=activation))
model.add(MaxPooling2D(pool_size=pool_size, strides=pool_strides))

model.add(Conv2D(filters=n_conv_neurons[1],
                 kernel_size=k_size, padding=padding, activation=activation))
model.add(MaxPooling2D(pool_size=pool_size, strides=pool_strides))

model.add(Conv2D(filters=n_conv_neurons[2],
                 kernel_size=k_size, padding=padding, activation=activation))
model.add(MaxPooling2D(pool_size=pool_size, strides=pool_strides))

model.add(Flatten())

model.add(Dense(units=n_dense_neurons[0], activation=activation))
model.add(Dense(units=n_dense_neurons[1], activation=activation))
model.add(Dense(units=n_dense_neurons[2], activation='softmax'))

predictions = model(x)

print("Output : ", predictions.shape)

Input :  (4, 28, 28, 3)
Output :  (4, 10)


for loop을 사용해서 조금 더 간단하게 작성하는 방법

In [21]:
model = Sequential()

for n_conv_neuron in n_conv_neurons:
    model.add(Conv2D(filters=n_conv_neuron,
                     kernel_size=k_size, padding=padding, activation=activation))
    model.add(MaxPooling2D(pool_size=pool_size, strides=pool_strides))

model.add(Flatten())

for n_dense_neuron in n_dense_neurons:
    model.add(Dense(units=n_dense_neuron, activation=activation))

# 마지막 dense layer는 softmax로 변경
model.add(Dense(units=n_dense_neurons[-1], activation='softmax'))

predictions = model(x)

print("Output : ", predictions.shape)

Output :  (4, 10)


## 2.2 Implementation with Model Sub-classing

연구, 개발 시 매우 많이 사용되는 방법

In [27]:
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [29]:
class TestCNN(Model):
    def __init__(self):
        super(TestCNN, self).__init__()
        
        # Feature Extractor 구현
        self.conv1 = Conv2D(filters=n_conv_neurons[0],
                            kernel_size=k_size, padding=padding, activation=activation)
        self.conv1_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)
        
        self.conv2 = Conv2D(filters=n_conv_neurons[1],
                            kernel_size=k_size, padding=padding, activation=activation)
        self.conv2_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

        self.conv3 = Conv2D(filters=n_conv_neurons[2],
                            kernel_size=k_size, padding=padding, activation=activation)
        self.conv3_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)
        
        self.flatten = Flatten()
        
        self.dense1 = Dense(units=n_dense_neurons[0], activation=activation)
        self.dense2 = Dense(units=n_dense_neurons[1], activation=activation)
        self.dense3 = Dense(units=n_dense_neurons[2], activation='softmax')
    
    # prototype 생성 시에는 print문으로 중간 결과 모두 확인해 볼 것
    def call(self, x):
        print(x.shape)
        
        x = self.conv1(x)      # convolutional
        print(x.shape)
        x = self.conv1_pool(x) # pooling
        print(x.shape)
        
        x = self.conv2(x)
        print(x.shape)
        x = self.conv2_pool(x)
        print(x.shape)
        
        x = self.conv3(x)
        print(x.shape)
        x = self.conv3_pool(x)
        print(x.shape)
         
        x = self.flatten(x)  # flatten
        print(x.shape)
        
        x = self.dense1(x)   # dense
        print(x.shape)
        x = self.dense2(x)   # dense
        print(x.shape)
        x = self.dense3(x)   # dense : softmax
        print(x.shape)
        
        return x       

In [31]:
N, n_H, n_W, n_C = 4, 28, 28, 3
n_conv_neurons = [10, 20, 30]
n_dense_neurons = [50, 30, 10]
k_size = 3   # kernel size
padding = 'same'
pool_size, pool_strides = 2, 2
activation = 'relu'

x = tf.random.normal(shape=(N, n_H, n_W, n_C))

model = TestCNN()
y = model(x)

(4, 28, 28, 3)
(4, 28, 28, 10)
(4, 14, 14, 10)
(4, 14, 14, 20)
(4, 7, 7, 20)
(4, 7, 7, 30)
(4, 3, 3, 30)
(4, 270)
(4, 50)
(4, 30)
(4, 10)


## 2.3 Implementation with Sequential + Layer Sub-classing

위에서와 같이 코딩을 하면, 코드가 너무 길어져서 중간에 실수할 수도 있음
- Layer Sub-classing 사용하는 게 좋음
  - Model sub-classing을 하는 것처럼 Layer Sub-classing을 하는 것
    - 앞에서 배운 것과 형태는 동일
    - 모델 fit 등에서 차이가 조금 있음

In [38]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer, Conv2D, MaxPooling2D, Flatten, Dense

In [45]:
class MyConv(Layer):
    def __init__(self, n_neuron):
        super(MyConv, self).__init__()
        
        self.conv = Conv2D(filters=n_neuron,
                           kernel_size=k_size, padding=padding, activation=activation)
        self.conv_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)
    
    def call(self, x):
        x = self.conv(x)
        x = self.conv_pool(x)
        return x

custom layer 사용해서 sequential 모델 만들기

In [46]:
N, n_H, n_W, n_C = 4, 28, 28, 3
n_conv_neurons = [10, 20, 30]
n_dense_neurons = [50, 30, 10]
k_size = 3   # kernel size
padding = 'same'
pool_size, pool_strides = 2, 2
activation = 'relu'

x = tf.random.normal(shape=(N, n_H, n_W, n_C))

In [48]:
model = Sequential()
model.add(MyConv(n_conv_neurons[0]))
model.add(MyConv(n_conv_neurons[1]))
model.add(MyConv(n_conv_neurons[2]))
model.add(Flatten())

model.add(Dense(units=n_dense_neurons[0], activation=activation))
model.add(Dense(units=n_dense_neurons[1], activation=activation))
model.add(Dense(units=n_dense_neurons[2], activation='softmax'))

predictions = model(x)
print("Output : ", predictions.shape)

Output :  (4, 10)


## 2.4 Implementation with Model and Layer Sub-classing

In [51]:
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, MaxPooling2D, Flatten, Dense

### 2.4.1 Custom Layer 생성

In [50]:
class MyConv(Layer):
    def __init__(self, n_neuron):
        super(MyConv, self).__init__()
        
        self.conv = Conv2D(filters=n_neuron,
                           kernel_size=k_size, padding=padding, activation=activation)
        self.conv_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)
    
    def call(self, x):
        x = self.conv(x)
        x = self.conv_pool(x)
        return x

### 2.4.2 Model 생성

In [ ]:
class TestCNN(Model):
    def __init__(self):
        super(TestCNN, self).__init__()
        
        self.conv1 = MyConv(n_conv_neurons[0])
        self.conv2 = MyConv(n_conv_neurons[1])
        self.conv3 = MyConv(n_conv_neurons[2])
        self.flatten = Flatten()
        
        self.dense1 = Dense(units=n_dense_neurons[0], activation=activation)
        self.dense2 = Dense(units=n_dense_neurons[1], activation=activation)
        self.dense3 = Dense(units=n_dense_neurons[2], activation='softmax')
    
    def call(self, x):

        x = self.conv1(x)      # convolutional
        x = self.conv2(x)
        x = self.conv3(x)

        x = self.flatten(x)  # flatten
        
        x = self.dense1(x)   # dense
        x = self.dense2(x)   # dense
        x = self.dense3(x)   # dense : softmax
        
        return x       

## 2.5 Sequential Model : 가장 효율적인 코드

In [2]:
import tensorflow as tf

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Layer, Conv2D, MaxPooling2D, Flatten, Dense

### 2.5.1 Custom Layer 생성

In [ ]:
class MyConv(Layer):
    def __init__(self, n_neuron):
        super(MyConv, self).__init__()
        
        self.conv = Conv2D(filters=n_neuron,
                           kernel_size=k_size, padding=padding, activation=activation)
        self.conv_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)
    
    def call(self, x):
        x = self.conv(x)
        x = self.conv_pool(x)
        return x

### 2.5.2 Model 생성

In [53]:
class TestCNN(Model):
    def __init__(self):
        super(TestCNN, self).__init__()
        
        # feature extractor를 sequential model로 생성
        self.fe = Seuqntial()
        
        # for n_neuron in n_conv_neurons:
            # self.fe.add(MyConv(n_neuron))
            
        self.fe.add(MyConv(n_conv_neurons[0]))
        self.fe.add(MyConv(n_conv_neurons[1]))
        self.fe.add(MyConv(n_conv_neurons[2]))
        self.fe.add(Flatten())
        
        self.classifier = Sequential()
        self.classifier.add(Dense(units=n_dense_neurons[0], activation=activation))
        self.classifier.add(Dense(units=n_dense_neurons[1], activation=activation))
        self.classifier.add(Dense(units=n_dense_neurons[2], activation='softmax'))
    
    def call(self, x):

        x = self.fe(x)
        #x = self.conv1(x)
        #x = self.conv2(x)
        #x = self.conv3(x)
        #x = self.flatten(x)
        
        x = self.classifier(x)
        #x = self.dense1(x)
        #x = self.dense2(x)
        #x = self.dense3(x)
        
        return x       

# 3. Implementation of LeNet

LeNet (르넷)
- CNN의 가장 classic한 model
- 사람의 손글씨를 구분하는 Neural Network (숫자 0~9 : 총 10개 구분)
- 처음에 Deep Learning을 유명하게 만들었던 model 중 하나

## 3.1 LeNet with Model Sub-classing

In [1]:
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense

In [8]:
class LeNet(Model):
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.conv1 = Conv2D(filters=6, kernel_size=5, padding='same', activation='tanh')
        self.conv1_pool = AveragePooling2D(pool_size=2, strides=2)
        
        self.conv2 = Conv2D(filters=16, kernel_size=5, padding='valid', activation='tanh')
        self.conv2_pool = AveragePooling2D(pool_size=2, strides=2)

        self.conv3 = Conv2D(filters=120, kernel_size=5, padding='valid', activation='tanh')
        
        self.flatten = Flatten()
        
        self.dense1 = Dense(units=84, activation='tanh')
        self.dense2 = Dense(units=10, activation='softmax')
    
    def call(self, x):
        print("x: {}".format(x.shape))
        
        x = self.conv1(x)
        print("x: {}".format(x.shape))
        x = self.conv1_pool(x)
        print("x: {}".format(x.shape))
        
        x = self.conv2(x)
        print("x: {}".format(x.shape))
        x = self.conv2_pool(x)
        print("x: {}".format(x.shape))
        
        x = self.conv3(x)
        print("x: {}".format(x.shape))
        
        x = self.flatten(x)
        print("x: {}".format(x.shape))
        
        x = self.dense1(x)
        print("x: {}".format(x.shape))
        x = self.dense2(x)
        print("x: {}".format(x.shape))
        
        return x        

### MNIST dataset은 흑백이미지 : channel = 1

In [9]:
model = LeNet()

x = tf.random.normal(shape=(32, 28, 28, 1))
predictions = model(x)

x: (32, 28, 28, 1)
x: (32, 28, 28, 6)
x: (32, 14, 14, 6)
x: (32, 10, 10, 16)
x: (32, 5, 5, 16)
x: (32, 1, 1, 120)
x: (32, 120)
x: (32, 84)
x: (32, 10)


## 3.2 LeNet with Hybrid Method

In [10]:
import tensorflow as tf

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Layer, Conv2D, AveragePooling2D, Flatten, Dense

In [11]:
class ConvLayer(Layer):
    def __init__(self, filters, padding, pool=True):
        super(ConvLayer, self).__init__()
        self.pool = pool        
        self.conv = Conv2D(filters=filters, kernel_size=5, padding=padding, activation='tanh')

        if self.pool == True:
            self.conv_pool = AveragePooling2D(pool_size=2, strides=2)
    
    
    def call(self, x):
        x = self.conv(x)
        if self.pool == True:
            x = self.conv_pool(x)
        return x
        

class LeNet(Model):
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.conv1 = ConvLayer(filters=6, padding='same')
        self.conv2 = ConvLayer(filters=16, padding='valid')
        self.conv3 = ConvLayer(filters=120, padding='valid', pool=False)
        
        self.flatten = Flatten()
        
        self.dense1 = Dense(units=84, activation='tanh')
        self.dense2 = Dense(units=10, activation='softmax')
    
    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        
        x = self.dense1(x)
        x = self.dense2(x)
        
        return x

In [12]:
model = LeNet()

x = tf.random.normal(shape=(32, 28, 28, 1))
predictions = model(x)

## 3.3 Forward Propagation of LeNet

In [25]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Layer, Conv2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.datasets import mnist  # 28×28×1 dataset
from tensorflow.keras.losses import SparseCategoricalCrossentropy

### 3.3.1 LeNet Implementation

In [26]:
class ConvLayer(Layer):
    def __init__(self, filters, padding, pool=True):
        super(ConvLayer, self).__init__()
        self.pool = pool        
        self.conv = Conv2D(filters=filters, kernel_size=5, padding=padding, activation='tanh')

        if self.pool == True:
            self.conv_pool = AveragePooling2D(pool_size=2, strides=2)
    
    
    def call(self, x):
        x = self.conv(x)
        if self.pool == True:
            x = self.conv_pool(x)
        return x
        

class LeNet(Model):
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.conv1 = ConvLayer(filters=6, padding='same')
        self.conv2 = ConvLayer(filters=16, padding='valid')
        self.conv3 = ConvLayer(filters=120, padding='valid', pool=False)
        
        self.flatten = Flatten()
        
        self.dense1 = Dense(units=84, activation='tanh')
        self.dense2 = Dense(units=10, activation='softmax')
    
    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        
        x = self.dense1(x)
        x = self.dense2(x)
        
        return x

### 3.3.2 Dataset Preparation

tensorflow에서 데이터 가져오기
- 원래 tensorflow data는 (train_images, train_labels), (test_images, test_labels)로 구성되어 있음
- 지금은 train 데이터만 받아서 사용하고자 함

In [27]:
(train_images, train_labels), _ = mnist.load_data()

6만 장의 데이터가 있음

In [28]:
print(train_images.shape, train_images.dtype)
print(train_labels.shape, train_labels.dtype)

(60000, 28, 28) uint8
(60000,) uint8


train_images
- 지금은 28×28 데이터인데 마지막 color 채널 추가하기
- dtype 변환 (uint8 → float32)

In [29]:
train_images = np.expand_dims(train_images, axis=3).astype(np.float32)
print(train_images.shape, train_images.dtype)

train_labels = train_labels.astype(np.int32)
print(train_labels.shape, train_labels.dtype)

(60000, 28, 28, 1) float32
(60000,) int32


지금 train_images와 train_labels는 numpy array
- tensor로 하나씩 꺼내 쓸 수 있도록 변환 : from_tensor_slices

In [30]:
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_ds = train_ds.batch(32)

### 3.3.3 Forward Propagation

In [31]:
model = LeNet()
loss_object = SparseCategoricalCrossentropy()

(32, 28, 28, 1) (32,)


한 바퀴만 돌려서 형태 확인
- 32장이 뽑히고 28×28×1 data임
- label : 최종 카테고리는 32개

In [ ]:
for images, labels in train_ds:
    print(images.shape, labels.shape)
    break

In [32]:
for images, labels in train_ds:
    predictions = model(images)
    loss = loss_object(labels, predictions)
    print(loss)
    
    break

tf.Tensor(2.3559756, shape=(), dtype=float32)
